# Spring Boot Generator in E2B Sandbox (Fixed Port Issues)

This notebook demonstrates how to generate and run a complete Spring Boot application in an E2B sandbox environment.

The notebook follows the official Spring Boot getting started tutorial and creates a web application with:
- Spring Boot 3.4.0
- Java 17
- Maven build system
- REST endpoint returning "Greetings from Spring Boot!"
- **Configurable port to avoid conflicts**

## Prerequisites

Make sure you have:
- E2B API key set in environment variable `E2B_API_KEY`
- Internet connection for downloading dependencies

## What this notebook does

1. **Setup**: Initialize E2B sandbox with Java 17 and Maven
2. **Port Check**: Verify port availability and find alternative if needed
3. **Project Creation**: Create complete Spring Boot project with custom port
4. **Environment Verification**: Check Java and Maven versions
5. **Build**: Compile the application with Maven
6. **Run**: Start the Spring Boot application
7. **Test**: Verify the application is working
8. **Cleanup**: Clean up resources

Let's get started! 🚀

## Install Required Packages

First, let's install the E2B Python SDK if it's not already available.

In [ ]:
%pip install e2b

## Import Libraries and Configuration

Import all the necessary libraries and configure the application port.

In [1]:
import os
import time
from e2b import Sandbox
from typing import List, Optional

# Configuration - Change APP_PORT if port conflicts occur
APP_PORT = 9090  # Default port for the Spring Boot application

def check_sandbox_port_availability(sandbox, port):
    """Check if a port is available in the sandbox"""
    print(f"🔍 Checking if port {port} is available in sandbox...")
    
    # Check if anything is listening on the port
    port_check = sandbox.commands.run(f"netstat -tlnp 2>/dev/null | grep :{port} || ss -tlnp | grep :{port} || echo 'Port {port} is free'", timeout=5)
    port_status = port_check.stdout.strip()
    
    if f":{port}" in port_status and ("LISTEN" in port_status.upper() or "listening" in port_status.lower()):
        print(f"⚠️  Port {port} is already in use in the sandbox")
        return False
    else:
        print(f"✅ Port {port} is available in the sandbox")
        return True

def find_available_port(sandbox, start_port=8080):
    """Find an available port starting from start_port"""
    port = start_port
    max_attempts = 10
    
    for i in range(max_attempts):
        if check_sandbox_port_availability(sandbox, port):
            return port
        port += 1
    
    print(f"❌ Could not find an available port after checking {max_attempts} ports")
    return None

print(f"📋 Application will use port: {APP_PORT}")
print("💡 Tip: Change APP_PORT above if you encounter port conflicts")

📋 Application will use port: 9090
💡 Tip: Change APP_PORT above if you encounter port conflicts


## Step 1: Setup and Initialization

Load the E2B API key and create a sandbox with the Spring Boot development template.

In [2]:
def step_1_setup_and_initialization():
    """Step 1: Setup and Initialization - Load API key and create sandbox"""
    print("🔧 Step 1: Setup and Initialization")

    # Load API key
    api_key = os.getenv('E2B_API_KEY')
    if not api_key:
        print("❌ E2B_API_KEY not found in environment variables")
        print("Please set your E2B API key:")
        print("export E2B_API_KEY='your-api-key-here'")
        return None

    print("🚀 Creating Spring Boot sandbox...")
    sandbox = Sandbox.create(template="springboot-dev", api_key=api_key)
    print("✅ Sandbox created successfully!")

    return sandbox

# Execute Step 1
sandbox = step_1_setup_and_initialization()

🔧 Step 1: Setup and Initialization
🚀 Creating Spring Boot sandbox...
✅ Sandbox created successfully!
✅ Sandbox created successfully!


## Step 2: Port Verification

Check if the configured port is available in the sandbox, and find an alternative if needed.

In [3]:
def step_2_verify_and_configure_port(sandbox, preferred_port):
    """Step 2: Port Verification - Check port availability and configure"""
    print(f"\n🔍 Step 2: Port Verification (Preferred: {preferred_port})")
    
    # First check the preferred port
    if check_sandbox_port_availability(sandbox, preferred_port):
        print(f"✅ Using preferred port: {preferred_port}")
        return preferred_port
    
    # If preferred port is not available, find another one
    print(f"⚠️  Preferred port {preferred_port} not available, searching for alternative...")
    available_port = find_available_port(sandbox, preferred_port + 1)
    
    if available_port:
        print(f"✅ Found available port: {available_port}")
        return available_port
    else:
        print("❌ No available ports found")
        return None

# Execute Step 2
configured_port = None
if sandbox:
    configured_port = step_2_verify_and_configure_port(sandbox, APP_PORT)


🔍 Step 2: Port Verification (Preferred: 9090)
🔍 Checking if port 9090 is available in sandbox...
✅ Port 9090 is available in the sandbox
✅ Using preferred port: 9090
✅ Port 9090 is available in the sandbox
✅ Using preferred port: 9090


## Step 3: Project Creation

Create the complete Spring Boot project structure with the configured port.

In [4]:
def step_3_create_project_structure(sandbox, port):
    """Step 3: Project Creation - Create directory structure and write files"""
    print(f"\n📁 Step 3: Project Creation (Port: {port})")

    # Create pom.xml exactly as in tutorial
    pom_content = '''<?xml version="1.0" encoding="UTF-8"?>
<project xmlns="http://maven.apache.org/POM/4.0.0" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
    xsi:schemaLocation="http://maven.apache.org/POM/4.0.0 https://maven.apache.org/xsd/maven-4.0.0.xsd">
    <modelVersion>4.0.0</modelVersion>

    <parent>
        <groupId>org.springframework.boot</groupId>
        <artifactId>spring-boot-starter-parent</artifactId>
        <version>3.4.0</version>
        <relativePath/>
    </parent>

    <groupId>com.example</groupId>
    <artifactId>spring-boot</artifactId>
    <version>0.0.1-SNAPSHOT</version>
    <name>spring-boot</name>
    <description>Demo project for Spring Boot</description>

    <properties>
        <java.version>17</java.version>
    </properties>

    <dependencies>
        <dependency>
            <groupId>org.springframework.boot</groupId>
            <artifactId>spring-boot-starter-web</artifactId>
        </dependency>

        <dependency>
            <groupId>org.springframework.boot</groupId>
            <artifactId>spring-boot-starter-test</artifactId>
            <scope>test</scope>
        </dependency>
    </dependencies>

    <build>
        <plugins>
            <plugin>
                <groupId>org.springframework.boot</groupId>
                <artifactId>spring-boot-maven-plugin</artifactId>
            </plugin>
        </plugins>
    </build>

</project>'''

    # Create Application.java exactly as in tutorial
    application_content = '''package com.example.springboot;

import java.util.Arrays;

import org.springframework.boot.CommandLineRunner;
import org.springframework.boot.SpringApplication;
import org.springframework.boot.autoconfigure.SpringBootApplication;
import org.springframework.context.ApplicationContext;
import org.springframework.context.annotation.Bean;

@SpringBootApplication
public class Application {

    public static void main(String[] args) {
        SpringApplication.run(Application.class, args);
    }

    @Bean
    public CommandLineRunner commandLineRunner(ApplicationContext ctx) {
        return args -> {

            System.out.println("Let's inspect the beans provided by Spring Boot:");

            String[] beanNames = ctx.getBeanDefinitionNames();
            Arrays.sort(beanNames);
            for (String beanName : beanNames) {
                System.out.println(beanName);
            }

        };
    }

}'''

    # Create HelloController.java exactly as in tutorial
    controller_content = '''package com.example.springboot;

import org.springframework.web.bind.annotation.GetMapping;
import org.springframework.web.bind.annotation.RestController;

@RestController
public class HelloController {

    @GetMapping("/")
    public String index() {
        return "Greetings from Spring Boot!";
    }

}'''

    # Create application.properties to configure port
    properties_content = f'''# Server configuration
server.port={port}

# Logging
logging.level.com.example=DEBUG
logging.level.org.springframework=INFO
'''

    # Create directory structure
    print("Creating project structure...")
    sandbox.commands.run("mkdir -p /home/user/spring-boot/src/main/java/com/example/springboot")
    sandbox.commands.run("mkdir -p /home/user/spring-boot/src/main/resources")
    sandbox.commands.run("mkdir -p /home/user/spring-boot/src/test/java/com/example/springboot")

    # Write files
    sandbox.files.write("/home/user/spring-boot/pom.xml", pom_content)
    sandbox.files.write("/home/user/spring-boot/src/main/java/com/example/springboot/Application.java", application_content)
    sandbox.files.write("/home/user/spring-boot/src/main/java/com/example/springboot/HelloController.java", controller_content)
    sandbox.files.write("/home/user/spring-boot/src/main/resources/application.properties", properties_content)

    print(f"✅ Project structure created successfully on port {port}!")

    return port

# Execute Step 3
if sandbox and configured_port:
    final_port = step_3_create_project_structure(sandbox, configured_port)


📁 Step 3: Project Creation (Port: 9090)
Creating project structure...
✅ Project structure created successfully on port 9090!
✅ Project structure created successfully on port 9090!


## Step 4: Environment Verification

Verify that Java 17 and Maven are properly installed and working in the sandbox.

In [5]:
def step_4_verify_environment(sandbox):
    """Step 4: Environment Verification - Check Java and Maven versions"""
    print("\n🔍 Step 4: Environment Verification")

    # Verify Java and Maven are working
    java_result = sandbox.commands.run("java -version")
    print("Java version:", java_result.stdout or java_result.stderr)

    maven_result = sandbox.commands.run("mvn -version")
    print("Maven version:", maven_result.stdout or maven_result.stderr)

    print("✅ Environment verification completed!")

# Execute Step 4
if sandbox:
    step_4_verify_environment(sandbox)


🔍 Step 4: Environment Verification
Java version: openjdk version "17.0.17" 2025-10-21
OpenJDK Runtime Environment (build 17.0.17+10-Debian-1deb12u1)
OpenJDK 64-Bit Server VM (build 17.0.17+10-Debian-1deb12u1, mixed mode, sharing)

Java version: openjdk version "17.0.17" 2025-10-21
OpenJDK Runtime Environment (build 17.0.17+10-Debian-1deb12u1)
OpenJDK 64-Bit Server VM (build 17.0.17+10-Debian-1deb12u1, mixed mode, sharing)

Maven version: Apache Maven 3.8.7
Maven home: /usr/share/maven
Java version: 17.0.17, vendor: Debian, runtime: /usr/lib/jvm/java-17-openjdk-amd64
Default locale: en_US, platform encoding: ANSI_X3.4-1968
OS name: "linux", version: "6.1.102", arch: "amd64", family: "unix"

✅ Environment verification completed!
Maven version: Apache Maven 3.8.7
Maven home: /usr/share/maven
Java version: 17.0.17, vendor: Debian, runtime: /usr/lib/jvm/java-17-openjdk-amd64
Default locale: en_US, platform encoding: ANSI_X3.4-1968
OS name: "linux", version: "6.1.102", arch: "amd64", family

## Step 5: Build Process

Build the Spring Boot application using Maven. This will download dependencies and compile the code.

In [6]:
def step_5_build_application(sandbox):
    """Step 5: Build Process - Build the application with streaming output"""
    print("\n🔨 Step 5: Build Process")

    def on_stdout(data):
        print(f"[BUILD] {data.strip()}")

    def on_stderr(data):
        print(f"[ERROR] {data.strip()}")

    try:
        build_result = sandbox.commands.run(
            "cd /home/user/spring-boot && mvn clean package -DskipTests",
            on_stdout=on_stdout,
            on_stderr=on_stderr,
            timeout=300
        )

        print(f"\nBuild completed with exit code: {build_result.exit_code}")

        if build_result.exit_code != 0:
            print("❌ Build failed!")
            print("Full stderr:", build_result.stderr)
            return False
        else:
            print("✅ Build successful!")
            return True

    except Exception as e:
        print(f"❌ Build error: {e}")
        # Try to see what's in the directory
        ls_result = sandbox.commands.run("cd /home/user/spring-boot && ls -la")
        print("Directory contents:", ls_result.stdout)
        return False

# Execute Step 5
build_success = False
if sandbox:
    build_success = step_5_build_application(sandbox)


🔨 Step 5: Build Process
[BUILD] [INFO] Scanning for projects...
[BUILD] [INFO] Scanning for projects...
[BUILD] Downloading from central: https://repo.maven.apache.org/maven2/org/springframework/boot/spring-boot-starter-parent/3.4.0/spring-boot-starter-parent-3.4.0.pom
[BUILD] Downloading from central: https://repo.maven.apache.org/maven2/org/springframework/boot/spring-boot-starter-parent/3.4.0/spring-boot-starter-parent-3.4.0.pom
[BUILD] Progress (1): 4.1 kB
[BUILD] Progress (1): 8.2 kB
[BUILD] Progress (1): 12 kB
[BUILD] Progress (1): 13 kB
[BUILD] 
[BUILD] Downloaded from central: https://repo.maven.apache.org/maven2/org/springframework/boot/spring-boot-starter-parent/3.4.0/spring-boot-starter-parent-3.4.0.pom (13 kB at 24 kB/s)
[BUILD] 
[BUILD] Downloading from central: https://repo.maven.apache.org/maven2/org/springframework/boot/spring-boot-dependencies/3.4.0/spring-boot-dependencies-3.4.0.pom
[BUILD] Progress (1): 4.1 kB
[BUILD] Progress (1): 8.2 kB
[BUILD] Progress (1): 12 kB

## Step 6: Check Build Artifacts

Verify that the build created the expected JAR file.

In [7]:
def step_6_check_build_artifacts(sandbox):
    """Step 6: Check Build Artifacts - Verify JAR file was created"""
    print("\n📦 Step 6: Check Build Artifacts")

    # Check JAR file
    jar_check = sandbox.commands.run("cd /home/user/spring-boot && ls -la target/*.jar")
    print("JAR files:")
    print(jar_check.stdout)

    print("✅ Build artifacts verified!")

# Execute Step 6
if sandbox and build_success:
    step_6_check_build_artifacts(sandbox)


📦 Step 6: Check Build Artifacts
JAR files:
-rw-r--r-- 1 user user 20655606 Oct 29 06:26 target/spring-boot-0.0.1-SNAPSHOT.jar

✅ Build artifacts verified!
JAR files:
-rw-r--r-- 1 user user 20655606 Oct 29 06:26 target/spring-boot-0.0.1-SNAPSHOT.jar

✅ Build artifacts verified!


## Step 7: Application Startup

Start the Spring Boot application in the background on the configured port.

In [8]:
def step_7_start_application(sandbox, port):
    """Step 7: Application Startup - Start the Spring Boot application"""
    print(f"\n🚀 Step 7: Application Startup (Port: {port})")

    jar_name = "target/spring-boot-0.0.1-SNAPSHOT.jar"
    start_cmd = f"cd /home/user/spring-boot && nohup java -jar {jar_name} > app.log 2>&1 &"
    print(f"Starting: {start_cmd}")

    # Start the app - use very short timeout since it should return immediately with &
    try:
        start_result = sandbox.commands.run(start_cmd, timeout=2)
        print("✅ Start command issued successfully")
        return True
    except Exception as e:
        print(f"⚠️  Start command may have timed out, but app might still be starting: {e}")
        return True  # Still return True since the app might be starting

# Execute Step 7
startup_success = False
if sandbox and build_success and 'final_port' in locals():
    startup_success = step_7_start_application(sandbox, final_port)


🚀 Step 7: Application Startup (Port: 9090)
Starting: cd /home/user/spring-boot && nohup java -jar target/spring-boot-0.0.1-SNAPSHOT.jar > app.log 2>&1 &
⚠️  Start command may have timed out, but app might still be starting: context deadline exceeded: This error is likely due to exceeding 'timeout' — the total time a long running request (like process or directory watch) can be active. It can be modified by passing 'timeout' when making the request. Use '0' to disable the timeout.
⚠️  Start command may have timed out, but app might still be starting: context deadline exceeded: This error is likely due to exceeding 'timeout' — the total time a long running request (like process or directory watch) can be active. It can be modified by passing 'timeout' when making the request. Use '0' to disable the timeout.


## Step 8: Startup Verification

Check if the application started properly by monitoring processes and the configured port.

In [9]:
def step_8_verify_startup(sandbox, port):
    """Step 8: Startup Verification - Check if application started properly"""
    print(f"\n🔍 Step 8: Startup Verification (Port: {port})")

    # Check multiple times with delays
    max_checks = 15  # Increased checks for slower startup
    process_found = False

    for i in range(max_checks):
        print(f"Check {i+1}/{max_checks}...")

        # Check if Java process is running
        ps_check = sandbox.commands.run("ps aux | grep java | grep -v grep", timeout=5)
        java_processes = ps_check.stdout.strip()
        if java_processes:
            print("✅ Java process found:")
            print(java_processes)
            process_found = True
            break
        else:
            print("⏳ No Java process yet, waiting...")
            time.sleep(3)

    # Check the configured port
    port_check = sandbox.commands.run(f"netstat -tlnp 2>/dev/null | grep :{port} || ss -tlnp | grep :{port} || echo 'Port {port} not listening'", timeout=5)
    port_status = port_check.stdout.strip()
    print(f"Port status: {port_status}")

    # Show recent logs
    print("\n📋 Recent application logs:")
    log_check = sandbox.commands.run("cd /home/user/spring-boot && cat app.log 2>/dev/null | tail -20 || echo 'No logs yet'", timeout=5)
    print(log_check.stdout)

    return process_found, port_status

# Execute Step 8
process_found = False
port_status = ""
if sandbox and build_success and startup_success and 'final_port' in locals():
    process_found, port_status = step_8_verify_startup(sandbox, final_port)


🔍 Step 8: Startup Verification (Port: 9090)
Check 1/15...
✅ Java process found:
user         607  0.0  0.1   3924  1640 ?        S    06:27   0:00 /bin/bash -l -c cd /home/user/spring-boot && nohup java -jar target/spring-boot-0.0.1-SNAPSHOT.jar > app.log 2>&1 &
user         608 27.0 12.4 2799820 125628 ?      Sl   06:27   0:07 java -jar target/spring-boot-0.0.1-SNAPSHOT.jar
Port status: LISTEN 0      100                *:9090             *:*    users:(("java",pid=608,fd=9))

📋 Recent application logs:
✅ Java process found:
user         607  0.0  0.1   3924  1640 ?        S    06:27   0:00 /bin/bash -l -c cd /home/user/spring-boot && nohup java -jar target/spring-boot-0.0.1-SNAPSHOT.jar > app.log 2>&1 &
user         608 27.0 12.4 2799820 125628 ?      Sl   06:27   0:07 java -jar target/spring-boot-0.0.1-SNAPSHOT.jar
Port status: LISTEN 0      100                *:9090             *:*    users:(("java",pid=608,fd=9))

📋 Recent application logs:
spring.task.execution-org.springframework

## Step 9: Endpoint Testing

Test the Spring Boot application's REST endpoint on the configured port.

In [10]:
def step_9_test_endpoints(sandbox, port, port_status):
    """Step 9: Endpoint Testing - Test the application endpoints"""
    print(f"\n🌐 Step 9: Endpoint Testing (Port: {port})")

    # Try to access the endpoint if port is listening
    if f"{port}" in port_status and "listening" in port_status.lower():
        try:
            curl_result = sandbox.commands.run(f"curl -m 5 http://localhost:{port}/ || echo 'Curl failed'", timeout=10)
            print("Endpoint response:")
            print(curl_result.stdout)
            return True
        except Exception as e:
            print(f"Curl test failed: {e}")
            return False
    else:
        print(f"⚠️  Port {port} not ready yet")
        return False

# Execute Step 9
endpoint_success = False
if sandbox and build_success and startup_success and 'final_port' in locals():
    endpoint_success = step_9_test_endpoints(sandbox, final_port, port_status)


🌐 Step 9: Endpoint Testing (Port: 9090)
⚠️  Port 9090 not ready yet


## Step 10: Cleanup

Clean up the sandbox resources.

In [11]:
def step_10_cleanup(sandbox):
    """Step 10: Cleanup - Kill the sandbox"""
    print("\n🔄 Step 10: Cleanup")

    if sandbox:
        sandbox.kill()
        print("✅ Sandbox killed")

# Execute Step 10
step_10_cleanup(sandbox)


🔄 Step 10: Cleanup
✅ Sandbox killed
✅ Sandbox killed


## Summary

Let's summarize what we accomplished in this notebook:

In [ ]:
# Final Summary
print("\n🎉 Spring Boot application setup completed!")
if 'final_port' in locals():
    print(f"📍 Application should be accessible at: http://localhost:{final_port}")
    print(f"🔗 Hello World endpoint: http://localhost:{final_port}/")
else:
    print("📍 Application port configuration not available")

if process_found and endpoint_success:
    print("✅ All steps completed successfully!")
    print("\n📋 What we built:")
    print("  • Complete Spring Boot 3.4.0 project")
    print("  • Java 17 application with Maven build")
    print("  • REST API with greeting endpoint")
    print("  • Embedded Tomcat server on configured port")
    print("  • Successfully running in E2B sandbox")
    if 'final_port' in locals():
        print(f"  • Port conflict resolution (using port {final_port})")
else:
    print("⚠️  Some steps may have issues - check logs above")
    print("\n🔧 Troubleshooting tips:")
    print("  • Check if the port is available in the sandbox")
    print("  • Verify the application logs for errors")
    print("  • Try changing the APP_PORT configuration")
    print("  • Ensure E2B_API_KEY is properly set")

print("\n💡 Next Steps:")
print("  • Try modifying the HelloController to add more endpoints")
print("  • Add database integration with Spring Data JPA")
print("  • Implement authentication and security")
print("  • Deploy to production environments")

## Complete Workflow (Run All Steps)

If you want to run the entire workflow in one go, uncomment and run the cell below:

In [12]:
# Uncomment the lines below to run the complete workflow
# This is equivalent to running the original script

def run_complete_workflow():
    sandbox = step_1_setup_and_initialization()
    if not sandbox:
        return
    
    configured_port = step_2_verify_and_configure_port(sandbox, APP_PORT)
    if not configured_port:
        step_10_cleanup(sandbox)
        return
    
    final_port = step_3_create_project_structure(sandbox, configured_port)
    step_4_verify_environment(sandbox)
    
    build_success = step_5_build_application(sandbox)
    if not build_success:
        step_10_cleanup(sandbox)
        return
    
    step_6_check_build_artifacts(sandbox)
    
    startup_success = step_7_start_application(sandbox, final_port)
    if not startup_success:
        step_10_cleanup(sandbox)
        return
    
    process_found, port_status = step_8_verify_startup(sandbox, final_port)
    endpoint_success = step_9_test_endpoints(sandbox, final_port, port_status)
    
    print("\n🎉 Complete workflow finished!")
    step_10_cleanup(sandbox)

# Run the complete workflow
run_complete_workflow()

🔧 Step 1: Setup and Initialization
🚀 Creating Spring Boot sandbox...
✅ Sandbox created successfully!

🔍 Step 2: Port Verification (Preferred: 9090)
🔍 Checking if port 9090 is available in sandbox...
✅ Sandbox created successfully!

🔍 Step 2: Port Verification (Preferred: 9090)
🔍 Checking if port 9090 is available in sandbox...
✅ Port 9090 is available in the sandbox
✅ Using preferred port: 9090

📁 Step 3: Project Creation (Port: 9090)
Creating project structure...
✅ Port 9090 is available in the sandbox
✅ Using preferred port: 9090

📁 Step 3: Project Creation (Port: 9090)
Creating project structure...
✅ Project structure created successfully on port 9090!

🔍 Step 4: Environment Verification
✅ Project structure created successfully on port 9090!

🔍 Step 4: Environment Verification
Java version: openjdk version "17.0.17" 2025-10-21
OpenJDK Runtime Environment (build 17.0.17+10-Debian-1deb12u1)
OpenJDK 64-Bit Server VM (build 17.0.17+10-Debian-1deb12u1, mixed mode, sharing)

Java version: